Install the required packages:

In [1]:
!pip install faiss-cpu accelerate torch langchain langchain_community ctransformers ctransformers[cuda]

In [3]:
import os
from langchain_community.llms import CTransformers
import faiss
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer

In [4]:
def embed_question(question, model_name):
    encoder = SentenceTransformer(model_name)
    embedding = encoder.encode([question])
    return embedding

def retrieve_indexes_from_faiss(embedding, k, database_name):
    # convert embeddings numpy float32 format for faiss:
    embedding_array = np.array(embedding, dtype=np.float32)
    index = faiss.read_index(database_name)

    # search faiss using query embedding:
    distances, indexes = index.search(embedding_array, k)
    indexes = indexes[0]
    distances = distances[0]
    print("indexes: ", indexes)
    print("distances: ", distances)
    return indexes

def retrieve_relevant_context_from_csv(indexes, csv_filename):
    df = pd.read_csv(csv_filename)
    context = ""
    for index in indexes:
        # get the text chunk where the id matches and add to context
        result = df.loc[df["id"] == index, "text_chunk"]
        if not result.empty:
            context = context + "\n" + result.values[0]
        else:
            print("no text chunk for id", index)

    print("retrieved context: ", context)
    return context

def construct_prompt(question, context):
    prompt_start = "Answer the following question: "
    promt_context = " Context: " + context
    prompt = prompt_start + question + promt_context
    return prompt


question= "what are the risk factors of developing schizophrenia?"
embedding = embed_question(question, "BAAI/bge-small-en-v1.5")
indexes = retrieve_indexes_from_faiss(embedding, 5, "faiss.index")
indexes = np.sort(indexes)
context = retrieve_relevant_context_from_csv(indexes, "text_chunks.csv")

# get the LM:
model_name = 'TheBloke/Orca-2-13B-GGUF'
os.environ['XDG_CACHE_HOME'] = 'drive/MyDrive/LLM_data/model/cache/'
config = {'temperature':0.2,'max_new_tokens': 1000, 'context_length':4000,'gpu_layers':50,'repetition_penalty':1 }
llm = CTransformers(model=model_name,
                    model_type="llama",
                    gpu_layers=50,
                    device = 0,
                    config=config,
                   )

prompt = construct_prompt(question, context)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


indexes:  [294 302 312 296 290]
distances:  [0.46032977 0.463894   0.4676258  0.4936387  0.5403944 ]
retrieved context:  
Negative symptoms are a lack of abilities associated with normal psychological functioning. Negative symptoms associated with schizophrenia include speech poverty and avolition.
Speech poverty is a reduction in the quality and amount of speech.
For example, a schizophrenic person may give one-word answers to questions without elaborating any further detail.
Avolition is a lack of desire and motivation for anything.
For example, a schizophrenic person may sit around without engaging in everyday tasks such as work, socialising, or maintaining personal hygiene.
Biological explanations of schizophrenia look at biological factors linked to schizophrenia. These include genetics, neural (brain) abnormalities, and abnormalities in dopamine.
The genetic explanation of schizophrenia looks at hereditary factors – i.e. genes – that contribute to the development of schizophrenia

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
# generate the response:
llm(prompt)

<ipython-input-5-88ec647f65ee>:2: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm(prompt)


'\n\nBased on the context provided, the risk factors of developing schizophrenia include:\n\n1. Genetic predisposition: The closer the genetic relationship to the person with schizophrenia, the greater the risk of developing schizophrenia. Identical twins have a 48% concordance rate for schizophrenia, while non-identical twins have a 17% concordance rate.\n\n2. Family dysfunction: High levels of conflict or difficulties in communication within the family can influence the development of schizophrenia.\n\n3. Environmental stressors: Exposure to environmental stressors, such as dysfunctional family upbringing, drug abuse, or other adverse life events, can trigger schizophrenia in individuals with a genetic predisposition.\n\n4. Neural abnormalities: Abnormalities in brain structures and dopamine levels have been observed in people with schizophrenia, suggesting a possible link between brain abnormalities and the development of the disorder.\n\n5. Dopamine abnormalities: Imbalances in dop